In [7]:
import rasterio
import geopandas as gpd
import numpy as np
from rasterio.mask import mask
from shapely.geometry import mapping

# Load the NUTS3 region shapefile (Ensure it contains the NUTS3 boundaries)
nuts3_gdf = gpd.read_file("NUTS_RG_03M_2024_4326.shp")  # Change to your actual file (e.g., .shp or .geojson)
print("Original CRS of NUTS3:", nuts3_gdf.crs)

# 🔹 Filter the specific NUTS3 region (Replace with actual region name or code)
selected_region = nuts3_gdf[nuts3_gdf["NUTS_ID"] == "PL523"]  # Replace with actual column name & NUTS3 code

# Check if the region exists
if selected_region.empty:
    raise ValueError("NUTS3 region not found! Check the NUTS3 code.")

# Reproject NUTS3 polygon to match raster CRS (ESRI:54009)
selected_region = selected_region.to_crs("ESRI:54009")
print("Reprojected CRS:", selected_region.crs)

# Load the population density raster (which is in ESRI:54009)
with rasterio.open("GHS_POP_E2020_GLOBE_R2023A_54009_100_V1_0.tif") as src:
    print("Raster CRS:", src.crs)

    # Clip the raster using the selected NUTS3 region
    out_image, out_transform = mask(src, [mapping(selected_region.geometry.iloc[0])], crop=True)

# Convert NoData values to NaN
out_image = np.where(out_image == src.nodata, np.nan, out_image)

# Calculate total population in the NUTS3 region
total_population = np.nansum(out_image)

print(f"Total Population in NUTS3 region: {total_population}")


Original CRS of NUTS3: EPSG:4326
Reprojected CRS: ESRI:54009
Raster CRS: ESRI:54009
Total Population in NUTS3 region: 364399.2259249864
